In [1]:
import os 
%pwd

'c:\\Users\\neha.mandwal\\.vscode\\Project\\MLOps\\NLP_text_summarization\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\neha.mandwal\\.vscode\\Project\\MLOps\\NLP_text_summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
        root_dir: Path
        data_path: Path
        tokenizer_name: Path

In [13]:
from src.NLP_text_summarization.utils.common import read_yaml, creating_directories
from src.NLP_text_summarization.constants import *

class ConfigurationManager:
    def __init__(self, config_path= CONFIG_FILE_PATH,
                params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        creating_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        creating_directories([self.config.root_dir])

        data_transformation_config =DataTransformationConfig(
                    root_dir= config.root_dir,
                    data_path= config.data_path,
                    tokenizer_name= config.tokenizer_name
            
        )
        return  data_transformation_config



In [7]:
import os
from src.NLP_text_summarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

c:\Users\neha.mandwal\.vscode\Project\MLOps\NLP_text_summarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer= Autotokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [15]:
        config = ConfigurationManager()
        data_transormation_config = config.get_data_transformation_config()
        data_transformation = DataTransformation(config=data_transformation_config)
        data_transformation.convert()
    

[2025-09-05 23:08:30,510: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-05 23:08:30,512: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-05 23:08:30,514: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'root_dir'"